In [145]:
# Mocking tenantProfiles
tenant_profiles = [
    {
        "_id": "11",
        "name": "Tenant 1",
        "description": "This is tenant 1",
        "tags": ["tag1", "tag2"],
        "requirements": [
            {
                "reqKey": "flattag3",
                "weight": 1
            },
            {
                "reqKey": "flattag4",
                "weight": 2
            }
        ],
        "images": ["image1.jpg", "image2.jpg"],
        "liked": ["f11"],
        "disliked": ["f22", "f33"],
    },{
        "_id": "22",
        "name": "Tenant 1",
        "description": "This is tenant 1",
        "tags": ["tag1", "tag2"],
        "requirements": [
            {
                "reqKey": "flattag1",
                "weight": 1
            },
            {
                "reqKey": "flattag2",
                "weight": 2
            }
        ],
        "images": ["image1.jpg", "image2.jpg"],
        "liked": [],
        "disliked": ["f11","f33"],
    },{
        "_id": "33",
        "name": "Tenant 1",
        "description": "This is tenant 1",
        "tags": ["tag1", "tag2"],
        "requirements": [
            {
                "reqKey": "flattag1",
                "weight": 1
            },
            {
                "reqKey": "flattag2",
                "weight": 2
            }
        ],
        "images": ["image1.jpg", "image2.jpg"],
        "liked": [],
        "disliked": [],
    },{
        "_id": "44",
        "name": "Tenant 1",
        "description": "This is tenant 1",
        "tags": ["tag1", "tag2"],
        "requirements": [
            {
                "reqKey": "flattag1",
                "weight": 1
            },
            {
                "reqKey": "flattag2",
                "weight": 2
            }
        ],
        "images": ["image1.jpg", "image2.jpg"],
        "liked": [],
        "disliked": [],
    },{
        "_id": "55",
        "name": "Tenant 1",
        "description": "This is tenant 1",
        "tags": ["tag3", "tag4"],
        "requirements": [
            {
                "reqKey": "flattag3",
                "weight": 1
            },
            {
                "reqKey": "flattag4",
                "weight": 2
            }
        ],
        "images": ["image1.jpg", "image2.jpg"],
        "liked": [],
        "disliked": [],
    },
    # Add more tenant profiles as needed
]

# Mocking flatProfiles
flat_profiles = [
    {
        "_id": "f11",
        "name": "Flat 1",
        "description": "This is flat 1",
        "tags": ["flattag3", "flattag4"],
        "requirements": [
            {
                "reqKey": "tag3",
                "weight": 1
            },
            {
                "reqKey": "tag4",
                "weight": 2
            }
        ],
        "address": "123 Street, City, Country",
        "price": 1000,
        "size": 100,
        "type": "apartment",
        "images": ["image1.jpg", "image2.jpg"],
        "approved": ["11"],
        "disapproved": ["22", "55"],
    },{
        "_id": "f22",
        "name": "Flat 1",
        "description": "This is flat 1",
        "tags": ["flattag1", "flattag2"],
        "requirements": [
            {
                "reqKey": "tag1",
                "weight": 1
            },
            {
                "reqKey": "tag2",
                "weight": 2
            }
        ],
        "address": "123 Street, City, Country",
        "price": 1000,
        "size": 100,
        "type": "apartment",
        "images": ["image1.jpg", "image2.jpg"],
        "approved": [],
        "disapproved": [],
    },{
        "_id": "f33",
        "name": "Flat 1",
        "description": "This is flat 1",
        "tags": ["flattag1", "flattag2"],
        "requirements": [
            {
                "reqKey": "tag1",
                "weight": 1
            },
            {
                "reqKey": "tag2",
                "weight": 2
            }
        ],
        "address": "123 Street, City, Country",
        "price": 1000,
        "size": 100,
        "type": "apartment",
        "images": ["image1.jpg", "image2.jpg"],
        "approved": [],
        "disapproved": [],
    }
    # Add more flat profiles as needed
]

In [146]:
available_flat_tags = ["flattag1", "flattag2", "flattag3", "flattag4"]
available_tenant_tags = ["tag1", "tag2", "tag3", "tag4", "tag5", "tag6", "tag7"]

flat_tag_mapping = {tag: index for index, tag in enumerate(available_flat_tags)}
tenant_tag_mapping = {tag: index for index, tag in enumerate(available_tenant_tags)}

beta = 0.3 # factor to control the importance of requirements of the other party

In [147]:
flat_index_mapping = {flat["_id"]: index for index, flat in enumerate(flat_profiles)}
tenant_index_mapping = {tenant["_id"]: index for index, tenant in enumerate(tenant_profiles)}

In [172]:
import numpy as np

flat_tag_matrix = np.zeros((len(flat_profiles), len(available_flat_tags)))
tenant_tag_matrix = np.zeros((len(tenant_profiles), len(available_tenant_tags)))

for index, flat in enumerate(flat_profiles):
    for tag in flat["tags"]:
        flat_tag_matrix[index, flat_tag_mapping[tag]] = 1
        
for index, tenant in enumerate(tenant_profiles):
    for tag in tenant["tags"]:
        tenant_tag_matrix[index, tenant_tag_mapping[tag]] = 1



In [149]:
print (flat_tag_matrix)

[[0. 0. 1. 1.]
 [1. 1. 0. 0.]
 [1. 1. 0. 0.]]


In [150]:
flat_requirements = np.zeros((len(flat_profiles), len(available_tenant_tags)))
tenant_requirements = np.zeros((len(tenant_profiles), len(available_flat_tags)))

for index, flat in enumerate(flat_profiles):
    for req in flat["requirements"]:
        flat_requirements[index, tenant_tag_mapping[req["reqKey"]]] = req["weight"]

for index, tenant in enumerate(tenant_profiles):
    for req in tenant["requirements"]:
        tenant_requirements[index, flat_tag_mapping[req["reqKey"]]] = req["weight"]


In [151]:
print (flat_requirements)

[[0. 0. 1. 2. 0. 0. 0.]
 [1. 2. 0. 0. 0. 0. 0.]
 [1. 2. 0. 0. 0. 0. 0.]]


In [152]:
match_flats_to_tenant_requirements = np.dot(flat_tag_matrix, tenant_requirements.T).T
match_tenants_to_flat_requirements = np.dot(tenant_tag_matrix, flat_requirements.T).T

def update_rankings():
    match_flats_to_tenant_requirements = np.dot(flat_tag_matrix, tenant_requirements.T).T
    match_tenants_to_flat_requirements = np.dot(tenant_tag_matrix, flat_requirements.T).T
    flat_ranking = match_flats_to_tenant_requirements + beta * match_tenants_to_flat_requirements.T
    tenant_ranking = match_tenants_to_flat_requirements + beta * match_flats_to_tenant_requirements.T

In [153]:
match_flats_to_tenant_requirements # tenant -> flat

array([[3., 0., 0.],
       [0., 3., 3.],
       [0., 3., 3.],
       [0., 3., 3.],
       [3., 0., 0.]])

In [154]:
match_tenants_to_flat_requirements # flat -> tenant

array([[0., 0., 0., 0., 3.],
       [3., 3., 3., 3., 0.],
       [3., 3., 3., 3., 0.]])

In [155]:
flat_ranking = match_flats_to_tenant_requirements + beta * match_tenants_to_flat_requirements.T
tenant_ranking = match_tenants_to_flat_requirements + beta * match_flats_to_tenant_requirements.T

In [156]:
flat_ranking

array([[3. , 0.9, 0.9],
       [0. , 3.9, 3.9],
       [0. , 3.9, 3.9],
       [0. , 3.9, 3.9],
       [3.9, 0. , 0. ]])

In [157]:
tenant_ranking

array([[0.9, 0. , 0. , 0. , 3.9],
       [3. , 3.9, 3.9, 3.9, 0. ],
       [3. , 3.9, 3.9, 3.9, 0. ]])

In [158]:
def get_best_flat_ids(tenant_id, n=10):
    indices = np.argsort(flat_ranking[tenant_index_mapping[tenant_id]])[::-1]
    return [flat_profiles[index]["_id"] for index in indices[:n]]

def get_best_tenants_ids(flat_id, n=10):
    indices = np.argsort(tenant_ranking[flat_index_mapping[flat_id]])[::-1]
    return [tenant_profiles[index]["_id"] for index in indices[:n]]



In [159]:
get_best_flat_ids("11")

['f11', 'f33', 'f22']

In [165]:
get_best_tenants_ids("f11")

['55', '11', '44', '33', '22']

## Filter and remove already liked, disliked and matched profiles

In [160]:
# Mocking liked, disliked and matched profiles

liked_flats = np.zeros((len(tenant_profiles), len(flat_profiles)), dtype=bool)
disliked_flats = np.zeros((len(tenant_profiles), len(flat_profiles)), dtype=bool)

for tenant_index, tenant in enumerate(tenant_profiles):
    for flat_id in tenant["liked"]:
        liked_flats[tenant_index, flat_index_mapping[flat_id]] = True
    for flat_id in tenant["disliked"]:
        disliked_flats[tenant_index, flat_index_mapping[flat_id]] = True

In [161]:
liked_flats

array([[ True, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [162]:
disliked_flats

array([[False,  True,  True],
       [ True, False,  True],
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [163]:
def get_liked_best_tenants_ids(flat_id, n=3):
    indices = np.argsort(tenant_ranking[flat_index_mapping[flat_id]] * liked_flats[:, flat_index_mapping[flat_id]])[::-1]
    disapproved_tenants = flat_profiles[flat_index_mapping[flat_id]]["disapproved"]
    best_tenants = []
    for index in indices:
        if tenant_profiles[index]["_id"] not in disapproved_tenants:
            best_tenants.append(tenant_profiles[index]["_id"])
    return best_tenants[:n]

In [164]:
get_liked_best_tenants_ids("f11")

['11', '44', '33']

In [166]:
def get_unseen_flat_ids(tenant_id, n=10):
    indices = np.argsort(flat_ranking[tenant_index_mapping[tenant_id]] * np.logical_not(liked_flats[tenant_index_mapping[tenant_id]]) * np.logical_not(disliked_flats[tenant_index_mapping[tenant_id]]))[::-1]
    return [flat_profiles[index]["_id"] for index in indices[:n]]

In [167]:
get_unseen_flat_ids("11")

['f33', 'f22', 'f11']

In [171]:
def like_flat(tenant_id, flat_id):
    liked_flats[tenant_index_mapping[tenant_id], flat_index_mapping[flat_id]] = True
    tenant_profiles[tenant_index_mapping[tenant_id]]["liked"].append(flat_id)
    if flat_id in tenant_profiles[tenant_index_mapping[tenant_id]]["disliked"]:
        tenant_profiles[tenant_index_mapping[tenant_id]]["disliked"].remove(flat_id)
    
def dislike_flat(tenant_id, flat_id):
    disliked_flats[tenant_index_mapping[tenant_id], flat_index_mapping[flat_id]] = True
    tenant_profiles[tenant_index_mapping[tenant_id]]["disliked"].append(flat_id)
    if flat_id in tenant_profiles[tenant_index_mapping[tenant_id]]["liked"]:
        tenant_profiles[tenant_index_mapping[tenant_id]]["liked"].remove(flat_id)
    
def approve_flat(flat_id, tenant_id):
    if tenant_id not in flat_profiles[flat_index_mapping[flat_id]]["liked"]:
        flat_profiles[flat_index_mapping[flat_id]]["approved"].append(tenant_id)
    if tenant_id in flat_profiles[flat_index_mapping[flat_id]]["disapproved"]:
        flat_profiles[flat_index_mapping[flat_id]]["disapproved"].remove(tenant_id)
    
def disapprove_flat(flat_id, tenant_id):
    if tenant_id not in flat_profiles[flat_index_mapping[flat_id]]["disliked"]:
        flat_profiles[flat_index_mapping[flat_id]]["disapproved"].append(tenant_id)
    if tenant_id in flat_profiles[flat_index_mapping[flat_id]]["approved"]:
        flat_profiles[flat_index_mapping[flat_id]]["approved"].remove(tenant_id)

In [169]:
like_flat("11", "f22")

In [170]:
get_unseen_flat_ids("11")

['f33', 'f22', 'f11']

In [174]:
def add_flat(flat):
    flat_profiles.append(flat)
    flat_index_mapping[flat["_id"]] = len(flat_profiles) - 1
    flat_tag_matrix = np.vstack((flat_tag_matrix, np.zeros((1, len(available_flat_tags)))))
    for tag in flat["tags"]:
        flat_tag_matrix[-1, flat_tag_mapping[tag]] = 1
    flat_requirements = np.vstack((flat_requirements, np.zeros((1, len(available_tenant_tags)))))
    for req in flat["requirements"]:
        flat_requirements[-1, tenant_tag_mapping[req["reqKey"]]] = req["weight"]
    update_rankings()
    # TODO update rankings more efficiently using only the newly added flat

def add_tenant(tenant):
    tenant_profiles.append(tenant)
    tenant_index_mapping[tenant["_id"]] = len(tenant_profiles) - 1
    tenant_tag_matrix = np.vstack((tenant_tag_matrix, np.zeros((1, len(available_tenant_tags)))))
    for tag in tenant["tags"]:
        tenant_tag_matrix[-1, tenant_tag_mapping[tag]] = 1
    tenant_requirements = np.vstack((tenant_requirements, np.zeros((1, len(available_flat_tags)))))
    for req in tenant["requirements"]:
        tenant_requirements[-1, flat_tag_mapping[req["reqKey"]]] = req["weight"]
    update_rankings()
    # TODO update rankings more efficiently using only the newly added tenant
    